In [1]:
import cv2
import torch
from pathlib import Path
from torchvision import transforms

# YOLOv5 ile oyuncu deteksiyonu
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

def detect_players(image):
    results = model(image)
    players = []
    for *box, conf, cls in results.xyxy[0]:
        if int(cls) == 0:  # 'person' class
            x1, y1, x2, y2 = map(int, box)
            torso = image[y1:y1 + (y2 - y1)//2, x1:x2]  # sadece gövde
            players.append(torso)
    return players


Using cache found in C:\Users\uysal/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2025-3-25 Python-3.9.21 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [2]:
!pip install pandas

In [25]:
import cv2
import numpy as np

def enhance_image(img):
    # Basit sharpen + resize + denoise
    img = cv2.resize(img, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)
    kernel = np.array([[0,-1,0], [-1,5,-1], [0,-1,0]])
    sharp = cv2.filter2D(img, -1, kernel)
    denoised = cv2.fastNlMeansDenoisingColored(sharp, None, 10, 10, 7, 21)
    return denoised


In [9]:
!!git clone https://github.com/baudm/parseq.git


["fatal: destination path 'parseq' already exists and is not an empty directory."]

In [10]:
import sys
sys.path.append('./parseq')  # parseq klasörün bulunduğu yer


In [13]:
import os
os.listdir('./parseq')



['.git',
 '.git-blame-ignore-revs',
 '.github',
 '.gitignore',
 '.pre-commit-config.yaml',
 'bench.py',
 'configs',
 'Datasets.md',
 'demo_images',
 'hubconf.py',
 'LICENSE',
 'Makefile',
 'NOTICE',
 'pyproject.toml',
 'read.py',
 'README.md',
 'requirements',
 'strhub',
 'test.py',
 'tools',
 'train.py',
 'tune.py']

In [15]:
!git clone https://github.com/abhishek4273/parseq.git parseq_real


^C


In [22]:
import pytesseract
import easyocr
from PIL import Image
import torch
from torchvision import transforms
from strhub.models.utils import load_from_checkpoint
from collections import Counter

# OCR okuyucuları başlat
easy_reader = easyocr.Reader(['en'])

# PARSeq yükleniyor
device = 'cuda' if torch.cuda.is_available() else 'cpu'
parseq = load_from_checkpoint('pretrained=parseq').eval().to(device)

# Görüntüyü PARSeq için hazırla
def prepare_image(img):
    if isinstance(img, str):  # path verilmişse
        img = Image.open(img).convert("RGB")
    elif isinstance(img, Image.Image):  # PIL image ise OK
        img = img.convert("RGB")
    else:
        raise ValueError("Girdi PIL Image ya da dosya yolu olmalı.")
    transform = transforms.ToTensor()
    return transform(img).unsqueeze(0).to(device)

# Jersey number okuma fonksiyonu
def read_jersey_number(img):
    results = []

    # EasyOCR
    easy_result = easy_reader.readtext(img)
    if easy_result:
        results.append(easy_result[0][1])

    # Tesseract
    tess_result = pytesseract.image_to_string(img, config='--psm 6 digits')
    if tess_result.strip():
        results.append(tess_result.strip())

    # PARSeq
    try:
        tensor_img = prepare_image(img)
        with torch.no_grad():
            logits = parseq(tensor_img)
            pred = logits.argmax(-1)
            text = parseq.tokenizer.decode(pred)[0]
            if text:
                results.append(text)
    except Exception as e:
        print("PARSeq hatası:", e)

    # Majority vote
    return Counter(results).most_common(1)[0][0] if results else "?"



C:\Users\uysal\anaconda3\envs\jerseyocr2\lib\site-packages\timm\models\helpers.py:7: FutureWarning: Importing from timm.models.helpers is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)


In [17]:
!pip install pytorch-lightning==1.9.5


  Using cached pytorch_lightning-1.9.5-py3-none-any.whl.metadata (23 kB)
  Using cached torchmetrics-1.7.0-py3-none-any.whl.metadata (21 kB)
  Using cached lightning_utilities-0.14.2-py3-none-any.whl.metadata (5.6 kB)
  Using cached aiohttp-3.11.14-cp39-cp39-win_amd64.whl.metadata (8.0 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.3.2-py2.py3-none-any.whl.metadata (3.8 kB)
  Using cached async_timeout-5.0.1-py3-none-any.whl.metadata (5.1 kB)
  Using cached frozenlist-1.5.0-cp39-cp39-win_amd64.whl.metadata (14 kB)
  Using cached multidict-6.2.0-cp39-cp39-win_amd64.whl.metadata (5.1 kB)
  Using cached yarl-1.18.3-cp39-cp39-win_amd64.whl.metadata (71 kB)
Using cached pytorch_lightning-1.9.5-py3-none-any.whl (829 kB)
Using cached lightning_utilities-0.14.2-py3-none-any.whl (28 kB)
Using cached torchmetrics-1.7.0-py3-none-any.whl (960 kB)
Using cached aiohttp-3.11.14-cp39-cp39-win_amd64.whl (443 kB)
Using cached aiohappyeyeballs-2.6.

In [19]:
!pip install nltk


  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached regex-2024.11.6-cp39-cp39-win_amd64.whl.metadata (41 kB)
Using cached nltk-3.9.1-py3-none-any.whl (1.5 MB)
Using cached regex-2024.11.6-cp39-cp39-win_amd64.whl (274 kB)
Using cached click-8.1.8-py3-none-any.whl (98 kB)
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)


In [21]:
!pip install timm


  Using cached timm-1.0.15-py3-none-any.whl.metadata (52 kB)
  Using cached safetensors-0.5.3-cp38-abi3-win_amd64.whl.metadata (3.9 kB)
Using cached timm-1.0.15-py3-none-any.whl (2.4 MB)
Using cached safetensors-0.5.3-cp38-abi3-win_amd64.whl (308 kB)


In [26]:
from matplotlib import pyplot as plt

image_path = r"C:\Users\uysal\OneDrive\Resimler\Ekran Görüntüleri\Ekran görüntüsü 2025-03-31 152531.png"
img = cv2.imread(image_path)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

players = detect_players(img_rgb)

for i, torso in enumerate(players):
    enhanced = enhance_image(torso)
    number = read_jersey_number(enhanced)
    
    plt.imshow(enhanced)
    plt.title(f'Player {i+1} - Jersey #: {number}')
    plt.axis('off')
    plt.show()


C:\Users\uysal/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:894: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


PARSeq hatası: Girdi PIL Image ya da dosya yolu olmalı.
PARSeq hatası: Girdi PIL Image ya da dosya yolu olmalı.
PARSeq hatası: Girdi PIL Image ya da dosya yolu olmalı.
PARSeq hatası: Girdi PIL Image ya da dosya yolu olmalı.
PARSeq hatası: Girdi PIL Image ya da dosya yolu olmalı.
PARSeq hatası: Girdi PIL Image ya da dosya yolu olmalı.
PARSeq hatası: Girdi PIL Image ya da dosya yolu olmalı.
PARSeq hatası: Girdi PIL Image ya da dosya yolu olmalı.
PARSeq hatası: Girdi PIL Image ya da dosya yolu olmalı.
PARSeq hatası: Girdi PIL Image ya da dosya yolu olmalı.
PARSeq hatası: Girdi PIL Image ya da dosya yolu olmalı.
PARSeq hatası: Girdi PIL Image ya da dosya yolu olmalı.
PARSeq hatası: Girdi PIL Image ya da dosya yolu olmalı.
PARSeq hatası: Girdi PIL Image ya da dosya yolu olmalı.
PARSeq hatası: Girdi PIL Image ya da dosya yolu olmalı.
PARSeq hatası: Girdi PIL Image ya da dosya yolu olmalı.
PARSeq hatası: Girdi PIL Image ya da dosya yolu olmalı.
PARSeq hatası: Girdi PIL Image ya da dosya yolu 

In [32]:
import os
import cv2
import torch
import pytesseract
import easyocr
import numpy as np
from collections import Counter
from matplotlib import pyplot as plt



In [33]:
easy_reader = easyocr.Reader(['en'])  # dil modeli


In [34]:
import subprocess

def parseq_predict(image_path):
    command = f"python parseq/test.py --checkpoint parseq/pretrained/parseq.pt --img {image_path} --topk 1"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    for line in result.stdout.splitlines():
        if "Prediction:" in line:
            return line.split("Prediction:")[1].strip()
    return None



In [35]:
def read_jersey_number(img, img_path_for_parseq):
    results = []

    # EasyOCR
    easy_result = easy_reader.readtext(img)
    if easy_result:
        results.append(easy_result[0][1])

    # Tesseract
    tess_result = pytesseract.image_to_string(img, config='--psm 6 digits')
    if tess_result.strip():
        results.append(tess_result.strip())

    # PARSeq
    parseq_result = parseq_predict(img_path_for_parseq)
    if parseq_result:
        results.append(parseq_result)

    return Counter(results).most_common(1)[0][0] if results else "?"



In [36]:
def enhance_image(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    enhanced = cv2.GaussianBlur(gray, (3,3), 0)
    return enhanced

def detect_players(img_rgb):
    # Yerine gerçek YOLO entegre edilecek
    h, w, _ = img_rgb.shape
    # Şimdilik rastgele crop örneği verelim (dummy bboxlar)
    crops = [img_rgb[50:150, 100:200], img_rgb[120:220, 250:350]]
    return crops


In [42]:
image_path = "Ekran görüntüsü 2025-03-31 152531.png"  # test görselin yolu
img = cv2.imread(image_path)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

players = detect_players(img_rgb)

for i, torso in enumerate(players):
    enhanced = enhance_image(torso)
    
    # PARSeq için görseli kaydet
    temp_path = f"temp_player_{i}.png"
    cv2.imwrite(temp_path, enhanced)
    success = cv2.imwrite(temp_path, enhanced)
    print(f"{temp_path} saved:", success)

    number = read_jersey_number(enhanced, temp_path)

    plt.imshow(enhanced, cmap='gray')
    plt.title(f"Player {i+1} - Jersey #: {number}")
    plt.axis('off')
    plt.show()

    os.remove(temp_path)  # temp dosyayı sil


temp_player_0.png saved: True
temp_player_1.png saved: True


In [43]:
import cv2
import matplotlib.pyplot as plt

# Görselin yolu
image_path = "temp_player_1.png"  # veya elindeki başka bir görselin yolu

# Görseli yükle
img = cv2.imread(image_path)

# BGR'den RGB'ye çevir (OpenCV BGR formatındadır)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Notebook'ta göster
plt.imshow(img_rgb)
plt.title("Görsel Önizleme")
plt.axis("off")
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: 'temp_player_1.png'

In [47]:
import os
import cv2
import torch
import pytesseract
import easyocr
import numpy as np
from PIL import Image
from collections import Counter
from torchvision import transforms
import matplotlib.pyplot as plt

# PARSeq yolunu ekle
import sys
sys.path.append('./parseq')

from strhub.models.utils import load_from_checkpoint


In [48]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

parseq_model = load_from_checkpoint('pretrained=parseq').eval().to(device)

# PARSeq için transform
parseq_transform = transforms.Compose([
    transforms.Resize((32, 128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])


In [49]:
easy_reader = easyocr.Reader(['en'])

def read_jersey_number(img_bgr, save_path):
    results = []

    # EasyOCR
    easy_result = easy_reader.readtext(img_bgr)
    if easy_result:
        results.append(easy_result[0][1])

    # Tesseract
    tess_result = pytesseract.image_to_string(img_bgr, config='--psm 6 digits')
    if tess_result.strip():
        results.append(tess_result.strip())

    # PARSeq
    img_pil = Image.open(save_path).convert('RGB')
    img_tensor = parseq_transform(img_pil).unsqueeze(0).to(device)
    parseq_result = parseq_model(img_tensor)[0]
    if parseq_result:
        results.append(parseq_result)

    return Counter(results).most_common(1)[0][0] if results else "?"


In [50]:
def detect_players(img_rgb):
    h, w, _ = img_rgb.shape
    torso1 = img_rgb[h//4:h//2, w//4:w//2]
    torso2 = img_rgb[h//4:h//2, w//2:w*3//4]
    return [torso1, torso2]


In [51]:
def enhance_image(img):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    return cv2.equalizeHist(gray)


In [53]:
image_path = "Ekran görüntüsü 2025-03-31 152531.png"
img = cv2.imread(image_path)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

players = detect_players(img_rgb)

for i, torso in enumerate(players):
    enhanced = enhance_image(torso)

    # Geçici olarak kaydet (PARSeq için)
    temp_path = f"temp_player_{i}.png"
    cv2.imwrite(temp_path, enhanced)

    number = read_jersey_number(enhanced, temp_path)

    # Görseli göster
    plt.imshow(enhanced, cmap='gray')
    plt.title(f"Player {i+1} - Jersey #: {number}")
    plt.axis('off')
    plt.show()

    os.remove(temp_path)

img = cv2.imread(image_path)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

players = detect_players(img_rgb)

for i, torso in enumerate(players):
    enhanced = enhance_image(torso)

    # Geçici olarak kaydet (PARSeq için)
    temp_path = f"temp_player_{i}.png"
    cv2.imwrite(temp_path, enhanced)

    number = read_jersey_number(enhanced, temp_path)

    # Görseli göster
    plt.imshow(enhanced, cmap='gray')
    plt.title(f"Player {i+1} - Jersey #: {number}")
    plt.axis('off')
    plt.show()

    os.remove(temp_path)


RuntimeError: Boolean value of Tensor with more than one value is ambiguous

In [54]:
# --- Gerekli kütüphaneler ---
import os
import cv2
import torch
import pytesseract
import easyocr
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
from collections import Counter
from parseq.strhub.models.utils import load_from_checkpoint

# --- PARSeq modeli yükleniyor ---
device = 'cuda' if torch.cuda.is_available() else 'cpu'
parseq_model = load_from_checkpoint('pretrained=parseq').eval().to(device)

# --- EasyOCR hazırla ---
easy_reader = easyocr.Reader(['en'])

# --- Oyuncu tespiti (placeholder) ---
def detect_players(image):
    # Bu, oyuncuları bulmak için YOLO gibi bir model kullanmalı
    # şu anda dummy olarak sadece tüm resmi döndürüyoruz
    return [image]

# --- Görsel iyileştirme ---
def enhance_image(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    resized = cv2.resize(gray, (128, 32))  # PARSeq için uygun boyut
    return resized

# --- Forma numarasını okuma ---
def read_jersey_number(img_bgr, save_path):
    results = []

    # EasyOCR
    easy_result = easy_reader.readtext(save_path)
    if len(easy_result) > 0:
        results.append(easy_result[0][1])

    # Tesseract
    tess_result = pytesseract.image_to_string(save_path, config='--psm 6 digits')
    if tess_result.strip():
        results.append(tess_result.strip())

    # PARSeq
    img_pil = Image.open(save_path).convert('RGB')
    img_tensor = parseq_model.transform(img_pil).unsqueeze(0).to(device)
    parseq_result = parseq_model(img_tensor)[0]
    if len(parseq_result) > 0:
        results.append(parseq_result)

    # Majority vote
    return Counter(results).most_common(1)[0][0] if results else "?"

# --- Test ---
image_path = "Ekran görüntüsü 2025-03-31 152531.png"  # Buraya test görselinin yolunu ver
img = cv2.imread(image_path)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

players = detect_players(img_rgb)

for i, torso in enumerate(players):
    enhanced = enhance_image(torso)

    # Geçici olarak kaydet (PARSeq için)
    temp_path = f"temp_player_{i}.png"
    cv2.imwrite(temp_path, enhanced)

    number = read_jersey_number(enhanced, temp_path)

    # Görseli göster
    plt.imshow(enhanced, cmap='gray')
    plt.title(f"Player {i+1} - Jersey #: {number}")
    plt.axis('off')
    plt.show()

    os.remove(temp_path)


AttributeError: 'PARSeq' object has no attribute 'transform'

In [59]:
# Gerekli kütüphaneler
import cv2
import numpy as np
import pytesseract
import easyocr
import torch
import os
from PIL import Image
from collections import Counter
from matplotlib import pyplot as plt
from parseq.strhub.models.utils import load_from_checkpoint
from parseq.strhub.data.module import SceneTextDataModule

# EasyOCR
easy_reader = easyocr.Reader(['en'])

# Cihaz
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# PARSeq modeli ve transform
parseq_model = load_from_checkpoint('pretrained=parseq').eval().to(device)
from parseq.utils import get_transform

transform = get_transform(img_size=(32, 128))


# Jersey numarası okuma fonksiyonu
def read_jersey_number(img_bgr, save_path):
    results = []

    # EasyOCR
    easy_result = easy_reader.readtext(img_bgr)
    if easy_result:
        results.append(easy_result[0][1])

    # Tesseract
    tess_result = pytesseract.image_to_string(img_bgr, config='--psm 6 digits')
    if tess_result.strip():
        results.append(tess_result.strip())

    # PARSeq
    img_pil = Image.open(save_path).convert('RGB')
    img_tensor = transform(image=img_pil)["image"].unsqueeze(0).to(device)
    parseq_result = parseq_model(img_tensor)[0]
    if len(parseq_result) > 0:
        results.append(parseq_result)

    return Counter(results).most_common(1)[0][0] if results else "?"

# Dummy torso tespit fonksiyonu (test için rastgele bölge seçer)
def detect_players(image_rgb):
    h, w, _ = image_rgb.shape
    players = []
    for i in range(3):  # Örnek olarak 3 oyuncu crop'luyoruz
        y1 = int(h * (0.2 + 0.2 * i))
        y2 = y1 + int(h * 0.15)
        x1 = int(w * 0.4)
        x2 = x1 + int(w * 0.2)
        torso = image_rgb[y1:y2, x1:x2]
        players.append(torso)
    return players

# Görüntü iyileştirme (isteğe bağlı basit enhance)
def enhance_image(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    enhanced = cv2.bilateralFilter(gray, 9, 75, 75)
    return enhanced

# Test başlat
def test_pipeline(image_path):
    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    players = detect_players(img_rgb)

    for i, torso in enumerate(players):
        enhanced = enhance_image(torso)

        temp_path = f"temp_player_{i}.png"
        cv2.imwrite(temp_path, enhanced)

        number = read_jersey_number(enhanced, temp_path)

        plt.imshow(enhanced, cmap='gray')
        plt.title(f"Player {i+1} - Jersey #: {number}")
        plt.axis('off')
        plt.show()

        os.remove(temp_path)

# Örnek test
test_pipeline("Ekran görüntüsü 2025-03-31 152531.png")  # Buraya test görselinin yolunu yaz

ModuleNotFoundError: No module named 'parseq.utils'

In [56]:
!pip install lmdb

  Using cached lmdb-1.6.2-cp39-cp39-win_amd64.whl.metadata (1.2 kB)
Using cached lmdb-1.6.2-cp39-cp39-win_amd64.whl (106 kB)
